In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd

In [56]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def plot_fluorescence_process(energy_gap=2.5, laser_wavelength=400):
    """
    Interactive visualization of fluorescence with Jablonski diagram and spectra
    
    Parameters:
    energy_gap - Energy difference between ground and excited states (eV)
    laser_wavelength - Wavelength of excitation laser (nm)
    """
    # Set static y-axis limits for Jablonski diagram
    y_min = -0.5
    y_max = 4.0
    
    # Energy levels
    ground_state = 0
    excited_state = energy_gap
    
    # Define min and max thresholds for vibrational energy levels
    vib_min_threshold = excited_state
    vib_max_threshold = excited_state + 0.3
    
    # Calculate the middle energy of the vibrational range
    vib_middle_energy = (vib_min_threshold + vib_max_threshold) / 2
    
    # Calculate wavelength corresponding to the middle of vibrational range
    # This will be the center of the absorption peak
    absorption_center = 1240 / (vib_middle_energy - ground_state - 0.1)
    
    # Calculate emission wavelength from energy gap
    # Exaggerate the Stokes shift by adding a larger offset
    emission_center = 1240 / energy_gap + 50  # Added +50nm to exaggerate Stokes shift
    
    # Laser wavelength in eV
    laser_energy = 1240 / laser_wavelength
    
    # Determine where laser hits on energy scale
    target_energy = ground_state + laser_energy
    
    # Determine if laser energy is within the vibrational energy range
    # Using the actual energy levels on the left plot for this check
    laser_in_range = vib_min_threshold -0.1 <= target_energy <= vib_max_threshold
    
    # Calculate emission intensity based on whether laser is in range
    if laser_in_range:
        # Relative intensity based on distance from middle of vibrational range
        absorption_efficiency = 1.0 - abs(target_energy - vib_middle_energy) / (vib_max_threshold - vib_min_threshold)
        emission_intensity = absorption_efficiency
    else:
        emission_intensity = 0.0
    
    # Set up the figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    #-----------------------
    # Jablonski Diagram (left subplot)
    #-----------------------
    
    # Create vibrational energy levels using np.linspace
    vib_count = 6  # Number of vibrational levels
    ground_vib_levels = np.linspace(ground_state + 0.05, ground_state + 0.3, vib_count)
    excited_vib_levels = np.linspace(vib_min_threshold, vib_max_threshold, vib_count)
    
    # Draw energy levels (main electronic states as thicker lines)
    ax1.hlines(ground_state, 1, 3, linewidth=2, color='blue')
    ax1.hlines(excited_state, 1, 3, linewidth=2, color='blue')
    
    # Draw vibrational levels
    for vl in ground_vib_levels:
        ax1.hlines(vl, 1.1, 2.9, linewidth=1, color='blue', alpha=0.5)
    
    for vl in excited_vib_levels:
        ax1.hlines(vl, 1.1, 2.9, linewidth=1, color='blue', alpha=0.5)
    
    # Gray out the Jablonski diagram if laser is out of range
    if not laser_in_range:
        # Gray rectangle over the whole diagram
        ax1.axhspan(y_min, y_max, alpha=0.3, color='gray')
        
        # Add text explaining why no fluorescence
        if target_energy > vib_max_threshold:
            message = "Laser energy too high\nNo absorption"
        else:
            message = "Laser energy too low\nNo absorption"
        
        ax1.text(2, (ground_state + excited_state)/2, message, 
                ha='center', va='center', color='red', fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'))
    
    # Always draw the absorption arrow to exactly target_energy
    if laser_in_range:
        # Draw absorption arrow to the exact laser energy level
        ax1.arrow(1.5, ground_state, 0, target_energy-ground_state, 
                  head_width=0.1, head_length=0.1, fc='purple', ec='purple', width=0.02)
        
        # Non-radiative relaxation from target to excited state baseline
        # If laser energy is below the excited state, draw relaxation upward
        if not target_energy < excited_state:
            ax1.plot([2, 2.3], [target_energy, excited_state], 
                    'k--', alpha=0.7)
        
        # Emission arrow - from excited state to a vibrational level of ground state
        ground_vib_level = ground_vib_levels[1]  # Choose a specific vibrational level
        ax1.arrow(2.5, excited_state, 0, 
                 -(excited_state-ground_vib_level), 
                 head_width=0.1, head_length=0.1, fc='red', ec='red', width=0.02)
    else:
        # Show laser arrow but with gray color to indicate it's not active
        ax1.arrow(1.5, ground_state, 0, target_energy-ground_state, 
                  head_width=0.1, head_length=0.1, fc='gray', ec='gray', width=0.02, 
                  alpha=0.5, linestyle='--')
    
    # Labels
    ax1.text(0.7, ground_state, "Ground State\nS₀", va='center')
    ax1.text(0.7, excited_state, "Excited State\nS₁", va='center')
    
    # Annotations for processes
    if laser_in_range:
        ax1.text(1.35, (ground_state + target_energy)/2, "Absorption\n(fs)", 
                color='purple', ha='right')
        ax1.text(2.15, (target_energy + excited_state)/2, "Vibrational\nRelaxation\n(ps)", 
                color='black', ha='center', alpha=0.7)
        ax1.text(2.7, (excited_state + ground_vib_level)/2, "Emission\n(ns)", 
                color='red', ha='left')
    
    # Set fixed y-axis limits for Jablonski diagram
    ax1.set_xlim(0.5, 3.5)
    ax1.set_ylim(y_min, y_max)
    ax1.set_title('Jablonski Diagram')
    ax1.set_ylabel('Energy (eV)')
    ax1.set_xticks([])
    
    #-----------------------
    # Absorption/Emission Spectra (right subplot)
    #-----------------------
    
    # Create wavelength range
    wavelengths = np.linspace(300, 700, 500)
    
    # Create absorption spectrum (narrower)
    absorption = np.exp(-(wavelengths - absorption_center)**2/(2*15**2))
    
    # Create emission spectrum (broader, red-shifted)
    emission_width = 15
    emission = np.exp(-(wavelengths - emission_center)**2/(2*emission_width**2))
    
    # Fix emission band to not cross the laser line
    if laser_in_range and laser_wavelength < emission_center:
        # Reduce emission intensity at wavelengths less than the laser wavelength
        cutoff_factor = np.ones_like(wavelengths)
        cutoff_idx = wavelengths < laser_wavelength
        cutoff_factor[cutoff_idx] = 0.2  # Reduce but don't eliminate
        emission = emission * cutoff_factor
    
    # Scale emission by absorption efficiency
    emission = emission * emission_intensity
    
    # Plot absorption and emission
    ax2.plot(wavelengths, absorption, 'b-', label='Absorption')
    ax2.plot(wavelengths, emission, 'r-', label='Emission')
    
    # Highlight the Stokes shift with an arrow
    if laser_in_range:
        # Draw an arrow connecting absorption and emission peaks
        stokes_shift = emission_center - absorption_center
        mid_y = 0.7  # Height of arrow
        ax2.annotate('', 
                   xy=(emission_center, mid_y), xytext=(absorption_center, mid_y),
                   arrowprops=dict(arrowstyle='<->', linewidth=2, color='green'))
        
        # Add a label for the Stokes shift
        ax2.text((absorption_center + emission_center)/2, mid_y+0.1, 
                f'Stokes Shift\n({stokes_shift:.0f} nm)', 
                ha='center', va='bottom', color='green',
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    
    # Plot laser line as vertical line
    ax2.axvline(x=laser_wavelength, color='purple', linestyle='-', linewidth=2, label='Laser')
    
    # Add text about laser
    if laser_in_range:
        ax2.text(laser_wavelength+5, 0.5, f"Laser: {laser_wavelength} nm", 
                color='purple', ha='left', va='center',
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    else:
        ax2.text(laser_wavelength+5, 0.5, f"Laser: {laser_wavelength} nm\n(No absorption)", 
                color='gray', ha='left', va='center',
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    
    # Add energy gap information
    energy_text = f"Energy Gap: {energy_gap:.2f} eV\nEmission: {emission_center:.0f} nm"
    ax2.text(max(wavelengths)-100, 0.9, energy_text, 
            ha='right', va='top', 
            bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    
    ax2.set_xlabel('Wavelength (nm)')
    ax2.set_ylabel('Normalized Intensity')
    ax2.set_title('Absorption and Emission Spectra')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(300, 700)
    ax2.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.show()

def interactive_fluorescence():
    """Create interactive widgets to control fluorescence visualization"""
    interact(
        plot_fluorescence_process,
        energy_gap=FloatSlider(min=1.8, max=3.5, step=0.1, value=2.5, 
                             description='Energy Gap (eV):'),
        laser_wavelength=FloatSlider(min=300, max=600, step=5, value=400, 
                                   description='Laser λ (nm):')
    )

In [57]:
interactive_fluorescence()

interactive(children=(FloatSlider(value=2.5, description='Energy Gap (eV):', max=3.5, min=1.8), FloatSlider(va…

In [62]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def plot_fluorescence_process(energy_gap=2.5, laser_wavelength=400):
    """
    Interactive visualization of fluorescence with Jablonski diagram and spectra
    
    Parameters:
    energy_gap - Energy difference between ground and excited states (eV)
    laser_wavelength - Wavelength of excitation laser (nm)
    """
    # Set static y-axis limits for Jablonski diagram
    y_min = -0.5
    y_max = 4.0
    
    # Energy levels
    ground_state = 0
    excited_state = energy_gap
    
    # Define min and max thresholds for vibrational energy levels
    vib_min_threshold = excited_state
    vib_max_threshold = excited_state + 0.3
    
    # Calculate the middle energy of the vibrational range
    vib_middle_energy = (vib_min_threshold + vib_max_threshold) / 2
    
    # Calculate wavelength corresponding to the middle of vibrational range
    # This will be the center of the absorption peak
    absorption_center = 1240 / (vib_middle_energy - ground_state - 0.1) - 15
    
    # Calculate emission wavelength from energy gap
    # Exaggerate the Stokes shift by adding a larger offset
    emission_center = 1240 / energy_gap + 50  # Added +50nm to exaggerate Stokes shift
    
    # Laser wavelength in eV
    laser_energy = 1240 / laser_wavelength
    
    # Determine where laser hits on energy scale
    target_energy = ground_state + laser_energy
    
    # Determine if laser energy is within the vibrational energy range
    # Using the actual energy levels on the left plot for this check
    laser_in_range = vib_min_threshold <= target_energy <= vib_max_threshold
    
    # Calculate emission intensity based on whether laser is in range
    if laser_in_range:
        # Relative intensity based on distance from middle of vibrational range
        absorption_efficiency = 1.0 - abs(target_energy - vib_middle_energy) / (vib_max_threshold - vib_min_threshold)
        emission_intensity = absorption_efficiency
    else:
        emission_intensity = 0.0
    
    # Set up the figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    #-----------------------
    # Jablonski Diagram (left subplot)
    #-----------------------
    
    # Create vibrational energy levels using np.linspace
    vib_count = 6  # Number of vibrational levels
    ground_vib_levels = np.linspace(ground_state + 0.05, ground_state + 0.3, vib_count)
    excited_vib_levels = np.linspace(vib_min_threshold, vib_max_threshold, vib_count)
    
    # Draw energy levels (main electronic states as thicker lines)
    ax1.hlines(ground_state, 1, 3, linewidth=2, color='blue')
    ax1.hlines(excited_state, 1, 3, linewidth=2, color='blue')
    
    # Draw vibrational levels
    for vl in ground_vib_levels:
        ax1.hlines(vl, 1.1, 2.9, linewidth=1, color='blue', alpha=0.5)
    
    for vl in excited_vib_levels:
        ax1.hlines(vl, 1.1, 2.9, linewidth=1, color='blue', alpha=0.5)
    
    # Gray out the Jablonski diagram if laser is out of range
    if not laser_in_range:
        # Gray rectangle over the whole diagram
        ax1.axhspan(y_min, y_max, alpha=0.3, color='gray')
        
        # Add text explaining why no fluorescence
        if target_energy > vib_max_threshold:
            message = "Laser energy too high\nNo absorption"
        else:
            message = "Laser energy too low\nNo absorption"
        
        ax1.text(2, (ground_state + excited_state)/2, message, 
                ha='center', va='center', color='red', fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'))
    
    # Always draw the absorption arrow to exactly target_energy
    if laser_in_range:
        # Draw absorption arrow to the exact laser energy level
        ax1.arrow(1.5, ground_state, 0, target_energy-ground_state, 
                  head_width=0.1, head_length=0.1, fc='purple', ec='purple', width=0.02)
        
        # Non-radiative relaxation from target to excited state baseline
        # If laser energy is below the excited state, draw relaxation upward
        if target_energy < excited_state:
            ax1.plot([2, 2.3], [target_energy, excited_state], 
                    'k--', alpha=0.7)
        # If laser energy is above the excited state, draw relaxation downward
        else:
            ax1.plot([2, 2.3], [target_energy, excited_state], 
                    'k--', alpha=0.7)
        
        # Emission arrow - from excited state to a vibrational level of ground state
        ground_vib_level = ground_vib_levels[1]  # Choose a specific vibrational level
        ax1.arrow(2.5, excited_state, 0, 
                 -(excited_state-ground_vib_level), 
                 head_width=0.1, head_length=0.1, fc='red', ec='red', width=0.02)
    else:
        # Show laser arrow but with gray color to indicate it's not active
        ax1.arrow(1.5, ground_state, 0, target_energy-ground_state, 
                  head_width=0.1, head_length=0.1, fc='gray', ec='gray', width=0.02, 
                  alpha=0.5, linestyle='--')
    
    # Labels
    ax1.text(0.7, ground_state, "Ground State\nS₀", va='center')
    ax1.text(0.7, excited_state, "Excited State\nS₁", va='center')
    
    # Annotations for processes
    if laser_in_range:
        ax1.text(1.35, (ground_state + target_energy)/2, "Absorption\n(fs)", 
                color='purple', ha='right')
        ax1.text(2.15, (target_energy + excited_state)/2, "Vibrational\nRelaxation\n(ps)", 
                color='black', ha='center', alpha=0.7)
        ax1.text(2.7, (excited_state + ground_vib_level)/2, "Emission\n(ns)", 
                color='red', ha='left')
    
    # Set fixed y-axis limits for Jablonski diagram
    ax1.set_xlim(0.5, 3.5)
    ax1.set_ylim(y_min, y_max)
    ax1.set_title('Jablonski Diagram')
    ax1.set_ylabel('Energy (eV)')
    ax1.set_xticks([])
    
    #-----------------------
    # Absorption/Emission Spectra (right subplot)
    #-----------------------
    
    # Create wavelength range
    wavelengths = np.linspace(300, 700, 500)
    
    # Create absorption spectrum (narrower)
    absorption = np.exp(-(wavelengths - absorption_center)**2/(2*15**2))
    
    # Create emission spectrum (broader, red-shifted)
    emission_width = 15
    emission = np.exp(-(wavelengths - emission_center)**2/(2*emission_width**2))
    
    # Fix emission band to not cross the laser line
    if laser_in_range and laser_wavelength < emission_center:
        # Reduce emission intensity at wavelengths less than the laser wavelength
        cutoff_factor = np.ones_like(wavelengths)
        cutoff_idx = wavelengths < laser_wavelength
        cutoff_factor[cutoff_idx] = 0.2  # Reduce but don't eliminate
        emission = emission * cutoff_factor
    
    # Scale emission by absorption efficiency
    emission = emission * emission_intensity
    
    # Plot absorption and emission
    ax2.plot(wavelengths, absorption, 'b-', label='Absorption')
    ax2.plot(wavelengths, emission, 'r-', label='Emission')
    
    # Plot laser line as vertical line
    ax2.axvline(x=laser_wavelength, color='purple', linestyle='-', linewidth=2, label='Laser')
    
    # Add text about laser
    if laser_in_range:
        ax2.text(laser_wavelength+5, 0.5, f"Laser: {laser_wavelength} nm", 
                color='purple', ha='left', va='center',
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    else:
        ax2.text(laser_wavelength+5, 0.5, f"Laser: {laser_wavelength} nm\n(No absorption)", 
                color='gray', ha='left', va='center',
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    
    # Add disclaimer about scale
    ax2.text(500, 0.05, "Note: Axes not to scale.\nStokes shift exaggerated for visualization.", 
             ha='center', va='bottom', style='italic', fontsize=9,
             bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.3'))
    
    ax2.set_xlabel('Wavelength (nm)')
    ax2.set_ylabel('Normalized Intensity')
    ax2.set_title('Absorption and Emission Spectra')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(300, 700)
    ax2.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.show()

def interactive_fluorescence():
    """Create interactive widgets to control fluorescence visualization"""
    interact(
        plot_fluorescence_process,
        energy_gap=FloatSlider(min=1.8, max=3.5, step=0.1, value=2.5, 
                             description='Energy Gap (eV):'),
        laser_wavelength=FloatSlider(min=300, max=600, step=5, value=400, 
                                   description='Laser λ (nm):')
    )

In [63]:
interactive_fluorescence()

interactive(children=(FloatSlider(value=2.5, description='Energy Gap (eV):', max=3.5, min=1.8), FloatSlider(va…